In [1]:
import os.path
import sys
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, InputLayer, BatchNormalization, ReLU, Softmax
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import to_categorical
import time

2024-03-26 01:09:24.020580: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def create_model(classes=2, input_size=150):
    input_shape = (input_size,)
    # Create model.
    model = Sequential()
    model.add(InputLayer(input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Dense(4, kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(ReLU())

    model.add(Dense(2, kernel_initializer='he_uniform'))
    model.add(BatchNormalization())
    model.add(ReLU())

    model.add(Dense(classes, kernel_initializer='he_uniform'))
    model.add(Softmax())

    optimizer = Nadam(learning_rate=0.001, epsilon=1e-08)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    # print(model.summary())
    return model

In [3]:
def check_file_exists(file_path):
    if not os.path.exists(file_path):
        print("Error: provided file path '%s' does not exist!" % file_path)
        sys.exit(-1)
    return

In [4]:
def load_traces(n=5):
    # Import our traces
    # Choose here which files to use
    traces = np.load(data_folder + 'trace.npy')
    labels = np.load(data_folder + 'label.npy')
    for i in range(1, n):
        tempTrace = np.load(data_folder + 'trace' + str(i) + '.npy')
        tempLabel = np.load(data_folder + 'label' + str(i) + '.npy')
        traces = np.append(traces, tempTrace, axis=0)
        labels = np.append(labels, tempLabel, axis=0)

    print('traces shape:', traces.shape)
    print('labels shape:', labels.shape)

    # Scale (standardize) traces
    delimitedTraces = np.zeros(traces.shape)
    for x_index in range(traces.shape[0]):
        delimitedTraces[x_index, :] = -1 + (traces[x_index, :] - np.min(traces[x_index, :])) * 2 / (
                    np.max(traces[x_index, :]) - np.min(traces[x_index, :]))

    return delimitedTraces, labels

In [5]:
def train_model(X_profiling, Y_profiling, model, save_file_name, epochs=50, batch_size=64):
    check_file_exists(os.path.dirname(save_file_name))
    # Save model every epoch
    save_model = ModelCheckpoint(save_file_name + '.keras', monitor='val_accuracy', verbose=1, save_best_only=True,
                                  mode='max')
    # Get the input layer shape
    input_layer_shape = model.input_shape
    # Sanity check
    if input_layer_shape[1] != len(X_profiling[0]):
        print("Error: model input shape %d instead of %d is not expected ..." % (input_layer_shape[1], len(X_profiling[0])))
        sys.exit(-1)
    # Adapt the data shape according our model input
    if len(input_layer_shape) == 2:
        # This is a MLP
        Reshaped_X_profiling = X_profiling
    elif len(input_layer_shape) == 3:
        # This is a CNN: expand the dimensions
        Reshaped_X_profiling = X_profiling.reshape((X_profiling.shape[0], X_profiling.shape[1], 1))
    else:
        print("Error: model input shape length %d is not expected ..." % len(input_layer_shape))
        sys.exit(-1)

    es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=20)

    history = model.fit(x=Reshaped_X_profiling, y=to_categorical(Y_profiling, num_classes=2), batch_size=batch_size,
                        verbose=0, epochs=epochs, callbacks=[es, save_model], validation_split=0.3)

    # Uncomment this to see how accuracy changes per epoch
    # # summarize history for accuracy
    # plt.plot(history.history['accuracy'])
    # plt.plot(history.history['val_accuracy'])
    # plt.title('model accuracy')
    # plt.ylabel('accuracy')
    # plt.xlabel('epoch')
    # plt.legend(['train', 'test'], loc='upper left')
    # # plt.savefig('../history/puf0/' + modelName + '.pdf')
    # plt.show()

    return history

In [6]:
data_folder = "../traces/puf12_avg100/training/"
model_folder = "../models/puf12/"
modelName = 'puf12_1'

# Start of execution, the time parts are there for our own references so we know roughly how long training takes
start = time.time()

# Load the training traces
traces, labels = load_traces(3)

### MLP training

mlp = create_model()

train_model(traces, labels, mlp, model_folder + modelName, epochs=100, batch_size=128)

end = time.time()
print("The total running time was: ", ((end - start) / 60), " minutes.")

traces shape: (30000, 150)
labels shape: (30000,)


/Users/william/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(



Epoch 1: val_accuracy improved from -inf to 0.59867, saving model to ../models/puf12/puf12_1.keras

Epoch 2: val_accuracy improved from 0.59867 to 0.95167, saving model to ../models/puf12/puf12_1.keras

Epoch 3: val_accuracy improved from 0.95167 to 0.97222, saving model to ../models/puf12/puf12_1.keras

Epoch 4: val_accuracy improved from 0.97222 to 0.97467, saving model to ../models/puf12/puf12_1.keras

Epoch 5: val_accuracy improved from 0.97467 to 0.97556, saving model to ../models/puf12/puf12_1.keras

Epoch 6: val_accuracy improved from 0.97556 to 0.97611, saving model to ../models/puf12/puf12_1.keras

Epoch 7: val_accuracy did not improve from 0.97611

Epoch 8: val_accuracy did not improve from 0.97611

Epoch 9: val_accuracy did not improve from 0.97611

Epoch 10: val_accuracy did not improve from 0.97611

Epoch 11: val_accuracy did not improve from 0.97611

Epoch 12: val_accuracy did not improve from 0.97611

Epoch 13: val_accuracy did not improve from 0.97611

Epoch 14: val_ac

In [7]:
data_folder = "../traces/puf0_avg100/training/"
model_folder = "../models/puf0/"
modelName = 'puf0_1'

# Start of execution, the time parts are there for our own references so we know roughly how long training takes
start = time.time()

# Load the training traces
traces, labels = load_traces(7)

### MLP training

mlp = create_model()

train_model(traces, labels, mlp, model_folder + modelName, epochs=100, batch_size=128)

end = time.time()
print("The total running time was: ", ((end - start) / 60), " minutes.")

traces shape: (70000, 150)
labels shape: (70000,)

Epoch 1: val_accuracy improved from -inf to 0.58938, saving model to ../models/puf0/puf0_1.keras

Epoch 2: val_accuracy improved from 0.58938 to 0.69614, saving model to ../models/puf0/puf0_1.keras

Epoch 3: val_accuracy improved from 0.69614 to 0.71886, saving model to ../models/puf0/puf0_1.keras

Epoch 4: val_accuracy improved from 0.71886 to 0.72319, saving model to ../models/puf0/puf0_1.keras

Epoch 5: val_accuracy improved from 0.72319 to 0.72543, saving model to ../models/puf0/puf0_1.keras

Epoch 6: val_accuracy improved from 0.72543 to 0.72795, saving model to ../models/puf0/puf0_1.keras

Epoch 7: val_accuracy did not improve from 0.72795

Epoch 8: val_accuracy did not improve from 0.72795

Epoch 9: val_accuracy improved from 0.72795 to 0.72857, saving model to ../models/puf0/puf0_1.keras

Epoch 10: val_accuracy did not improve from 0.72857

Epoch 11: val_accuracy did not improve from 0.72857

Epoch 12: val_accuracy did not impr